Neste modelo o custo de transporte é considerado por caminhao

In [438]:
# dados do problema

import pandas as pd

periodos = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Periods')
plantas = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Plants', decimal=",")
linhas = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Lines')
cds = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Dist. Centers', decimal=",")
clientes = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Customers', decimal=",")
produtos = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Products')
rotas = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Routes', decimal=",")
demandas = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Demands')
capabilidades = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Capabilities')
ktaxas = pd.read_excel('./sample_data/planilha_inicial.xlsx', sheet_name='Rates')

#Conjuntos

In [439]:
P = plantas['name'].tolist() # set plants
W = cds['name'].tolist() # set warehouses
Lines = linhas['name'].tolist() # set Line names
plants_of_lines = linhas['plant'].tolist() # linhas de cada planta

In [440]:
p_l = dict(zip(Lines,plants_of_lines))

k = {}
for _, i in ktaxas.iterrows():
  # var = i.to_dict()
  line, period, size, rate = i['line'], i['period'], i['size'], i['rate']
  k[(size,line,period)] = rate

C = clientes['name'].tolist() # clientes
I = produtos['product'].tolist() # produtos
E = produtos['size'].tolist() # tamanhos

T = periodos[0].tolist() #periodos


size_of_products = dict(zip(I,E)) #tamanho de cada produto
tamanhos = list(set(E))
products_per_size = {i:[] for i in tamanhos} # produtos por tamanho
for i in I:
  products_per_size[size_of_products[i]].append(i)


In [441]:
period_index = {}
print(T)
for i in range(len(T)):
  period_index[T[i]] = i

print(period_index)
periodos = list(period_index.keys())

[Timestamp('2025-05-14 00:00:00'), Timestamp('2025-05-15 00:00:00'), Timestamp('2025-05-16 00:00:00'), Timestamp('2025-05-17 00:00:00'), Timestamp('2025-05-18 00:00:00'), Timestamp('2025-05-19 00:00:00'), Timestamp('2025-05-20 00:00:00'), Timestamp('2025-05-21 00:00:00'), Timestamp('2025-05-22 00:00:00'), Timestamp('2025-05-23 00:00:00'), Timestamp('2025-05-24 00:00:00'), Timestamp('2025-05-25 00:00:00'), Timestamp('2025-05-26 00:00:00'), Timestamp('2025-05-27 00:00:00'), Timestamp('2025-05-28 00:00:00'), Timestamp('2025-05-29 00:00:00'), Timestamp('2025-05-30 00:00:00'), Timestamp('2025-05-31 00:00:00')]
{Timestamp('2025-05-14 00:00:00'): 0, Timestamp('2025-05-15 00:00:00'): 1, Timestamp('2025-05-16 00:00:00'): 2, Timestamp('2025-05-17 00:00:00'): 3, Timestamp('2025-05-18 00:00:00'): 4, Timestamp('2025-05-19 00:00:00'): 5, Timestamp('2025-05-20 00:00:00'): 6, Timestamp('2025-05-21 00:00:00'): 7, Timestamp('2025-05-22 00:00:00'): 8, Timestamp('2025-05-23 00:00:00'): 9, Timestamp('2025-

In [442]:
#relate products to sizes

def prod_size(prod=None, size=None):
  if prod != None:
    return size_of_products[prod]
  elif size != None:
    return products_per_size[size]

#Parametros

In [443]:
G_p = dict(zip(P, list(plantas['storage_capacity']))) # capacidade max das plantas
G_w = dict(zip(W, list(cds['storage_capacity']))) # capacidade max dos warehouses
taxas = k # taxas por linha linha plant0_1:{periodo_t:{size:12oz, rate:k}}
print(taxas)

{('9.1oz Sleek', 'Plant 0_1', Timestamp('2025-05-14 00:00:00')): 2745850, ('12oz', 'Plant 0_2', Timestamp('2025-05-14 00:00:00')): 2788956, ('12oz Sleek', 'Plant 0_3', Timestamp('2025-05-14 00:00:00')): 2882906, ('16oz', 'Plant 1_1', Timestamp('2025-05-14 00:00:00')): 3008418, ('12oz Sleek', 'Plant 1_2', Timestamp('2025-05-14 00:00:00')): 2855797, ('24oz', 'Plant 1_3', Timestamp('2025-05-14 00:00:00')): 3164050, ('24oz', 'Plant 2_1', Timestamp('2025-05-14 00:00:00')): 3474925, ('12oz', 'Plant 2_2', Timestamp('2025-05-14 00:00:00')): 2809202, ('9.1oz Sleek', 'Plant 0_1', Timestamp('2025-05-15 00:00:00')): 2745850, ('12oz', 'Plant 0_2', Timestamp('2025-05-15 00:00:00')): 2788956, ('12oz Sleek', 'Plant 0_3', Timestamp('2025-05-15 00:00:00')): 2882906, ('16oz', 'Plant 1_1', Timestamp('2025-05-15 00:00:00')): 3008418, ('12oz Sleek', 'Plant 1_2', Timestamp('2025-05-15 00:00:00')): 2855797, ('24oz', 'Plant 1_3', Timestamp('2025-05-15 00:00:00')): 3164050, ('24oz', 'Plant 2_1', Timestamp('2025

In [444]:
t_pc = {} # taxa de frete | indexado por t_pc[(planta_p, cliente_c)]
for _,i in rotas.iterrows():
  if i.origin[0] == "P" and i.destination[0] == "C":
    org, dest, cost = i.origin, i .destination, i.freight_cost
    t_pc[(org, dest)] = cost

print(t_pc)

{('Plant 0', 'Customer 3'): 19383.46, ('Plant 0', 'Customer 4'): 6438.017, ('Plant 0', 'Customer 5'): 14390.436, ('Plant 0', 'Customer 6'): 15371.846, ('Plant 0', 'Customer 7'): 12974.746, ('Plant 0', 'Customer 8'): 22674.023, ('Plant 0', 'Customer 9'): 6785.053, ('Plant 0', 'Customer 11'): 22960.098, ('Plant 0', 'Customer 12'): 706.936, ('Plant 1', 'Customer 0'): 6220.907, ('Plant 1', 'Customer 1'): 12124.033, ('Plant 1', 'Customer 2'): 9249.101, ('Plant 1', 'Customer 3'): 4473.001, ('Plant 1', 'Customer 4'): 13596.947, ('Plant 1', 'Customer 5'): 5235.362, ('Plant 1', 'Customer 6'): 7468.514, ('Plant 1', 'Customer 7'): 11060.915, ('Plant 1', 'Customer 8'): 3261.027, ('Plant 1', 'Customer 9'): 15540.957, ('Plant 1', 'Customer 10'): 13340.207, ('Plant 1', 'Customer 11'): 4304.48, ('Plant 1', 'Customer 12'): 18941.362, ('Plant 1', 'Customer 13'): 9984.768, ('Plant 1', 'Customer 14'): 4946.632, ('Plant 2', 'Customer 1'): 7732.661, ('Plant 2', 'Customer 2'): 5742.955, ('Plant 2', 'Customer

In [445]:
t_wc = {} # taxa de frete | indexado por t_wc[(warehouse_w, cliente_c)]
for _,i in rotas.iterrows():
  if i.origin[0] == "W" and i.destination[0] == "C":
    org, dest, cost = i.origin, i .destination, i.freight_cost
    t_wc[(org, dest)] = cost
print(t_wc)

{('Warehouse 1', 'Customer 3'): 19458.671, ('Warehouse 1', 'Customer 4'): 6478.39, ('Warehouse 1', 'Customer 5'): 14801.712, ('Warehouse 1', 'Customer 6'): 16080.879, ('Warehouse 1', 'Customer 7'): 12682.06, ('Warehouse 1', 'Customer 8'): 22960.251, ('Warehouse 1', 'Customer 9'): 7803.849, ('Warehouse 1', 'Customer 11'): 23429.833, ('Warehouse 1', 'Customer 12'): 1724.557, ('Warehouse 2', 'Customer 0'): 6727.265, ('Warehouse 3', 'Customer 0'): 4975.47, ('Warehouse 4', 'Customer 0'): 5369.729, ('Warehouse 2', 'Customer 1'): 11359.979, ('Warehouse 3', 'Customer 1'): 12367.644, ('Warehouse 4', 'Customer 1'): 13610.102, ('Warehouse 2', 'Customer 2'): 9240.758, ('Warehouse 3', 'Customer 2'): 8122.626, ('Warehouse 4', 'Customer 2'): 9388.724, ('Warehouse 2', 'Customer 3'): 4156.454, ('Warehouse 3', 'Customer 3'): 5756.143, ('Warehouse 4', 'Customer 3'): 5436.404, ('Warehouse 2', 'Customer 4'): 12816.566, ('Warehouse 3', 'Customer 4'): 13934.811, ('Warehouse 4', 'Customer 4'): 15114.024, ('Wa

In [446]:
t_pp = {} # taxa de frete | indexado por t_pp[(planta_p, planta_q)]
for _,i in rotas.iterrows():
  if i.origin[0] == "P" and i.destination[0] == "P":
    org, dest, cost = i.origin, i .destination, i.freight_cost
    t_pp[(org, dest)] = cost

print(t_pp)

{('Plant 0', 'Plant 1'): 19539.596, ('Plant 0', 'Plant 2'): 14434.546, ('Plant 1', 'Plant 0'): 19539.596, ('Plant 1', 'Plant 2'): 6191.956, ('Plant 2', 'Plant 0'): 14434.546, ('Plant 2', 'Plant 1'): 6191.956}


In [447]:
t_pw = {} # taxa de frete | indexado por t_pp[(planta_p, warehouse_w)]
for _,i in rotas.iterrows():
  if i.origin[0] == "P" and i.destination[0] == "W":
    org, dest, cost = i.origin, i .destination, i.freight_cost
    t_pw[(org, dest)] = cost

print(t_pw)

{('Plant 0', 'Warehouse 1'): 1135.435, ('Plant 0', 'Warehouse 2'): 18793.522, ('Plant 0', 'Warehouse 3'): 19663.53, ('Plant 0', 'Warehouse 4'): 20984.536, ('Plant 0', 'Warehouse 5'): 15166.715, ('Plant 1', 'Warehouse 1'): 19878.642, ('Plant 1', 'Warehouse 2'): 796.25, ('Plant 1', 'Warehouse 3'): 1285.983, ('Plant 1', 'Warehouse 4'): 1557.224, ('Plant 1', 'Warehouse 5'): 5159.554, ('Plant 2', 'Warehouse 1'): 15003.239, ('Plant 2', 'Warehouse 2'): 5704.666, ('Plant 2', 'Warehouse 3'): 5756.687, ('Plant 2', 'Warehouse 4'): 7256.083, ('Plant 2', 'Warehouse 5'): 1039.245}


In [448]:
# t_wp = {} # taxa de frete | indexado por t_wp[(warehouse_p, planta_q)]
# for _,i in rotas.iterrows():
#   if i.origin[0] == "W" and i.destination[0] == "P":
#     org, dest, cost = i.origin, i .destination, i.freight_cost
#     t_wp[(org, dest)] = cost

# print(t_wp)

In [449]:
ld = {} # leadtimes
for _, r in rotas.iterrows():
  origem, destino, leadtime = r['origin'], r['destination'], r['leadtime']
  ld[(origem, destino)] = leadtime
print(ld)

{('Plant 0', 'Warehouse 1'): 0, ('Warehouse 1', 'Plant 0'): 0, ('Plant 0', 'Warehouse 2'): 7, ('Warehouse 2', 'Plant 0'): 7, ('Plant 0', 'Warehouse 3'): 7, ('Warehouse 3', 'Plant 0'): 7, ('Plant 0', 'Warehouse 4'): 8, ('Warehouse 4', 'Plant 0'): 8, ('Plant 0', 'Warehouse 5'): 6, ('Warehouse 5', 'Plant 0'): 6, ('Plant 0', 'Plant 1'): 7, ('Plant 0', 'Plant 2'): 5, ('Plant 0', 'Customer 3'): 7, ('Warehouse 1', 'Customer 3'): 7, ('Plant 0', 'Customer 4'): 2, ('Warehouse 1', 'Customer 4'): 2, ('Plant 0', 'Customer 5'): 5, ('Warehouse 1', 'Customer 5'): 6, ('Plant 0', 'Customer 6'): 6, ('Warehouse 1', 'Customer 6'): 6, ('Plant 0', 'Customer 7'): 5, ('Warehouse 1', 'Customer 7'): 5, ('Plant 0', 'Customer 8'): 9, ('Warehouse 1', 'Customer 8'): 9, ('Plant 0', 'Customer 9'): 3, ('Warehouse 1', 'Customer 9'): 3, ('Plant 0', 'Customer 11'): 9, ('Warehouse 1', 'Customer 11'): 9, ('Plant 0', 'Customer 12'): 0, ('Warehouse 1', 'Customer 12'): 1, ('Plant 1', 'Warehouse 1'): 7, ('Warehouse 1', 'Plant 1

In [450]:
total_demand = 0
D = {} # Demandas | indexado por D[(customer_c, product_i, period_t)]
for _,r in demandas.iterrows():
  customer, prod, perio , demand = r['customer'], r['product'], r['period'], r['demand']
  D[(customer, prod, perio)] = demand
  total_demand+=demand
print(total_demand)
print(D)


213600000
{('Customer 0', 'Product 20', Timestamp('2025-05-25 00:00:00')): 1200000, ('Customer 0', 'Product 21', Timestamp('2025-05-25 00:00:00')): 800000, ('Customer 0', 'Product 15', Timestamp('2025-05-25 00:00:00')): 800000, ('Customer 1', 'Product 23', Timestamp('2025-05-25 00:00:00')): 1000000, ('Customer 1', 'Product 16', Timestamp('2025-05-25 00:00:00')): 600000, ('Customer 1', 'Product 15', Timestamp('2025-05-25 00:00:00')): 1000000, ('Customer 2', 'Product 24', Timestamp('2025-05-25 00:00:00')): 600000, ('Customer 2', 'Product 17', Timestamp('2025-05-25 00:00:00')): 1600000, ('Customer 3', 'Product 15', Timestamp('2025-05-25 00:00:00')): 800000, ('Customer 4', 'Product 2', Timestamp('2025-05-25 00:00:00')): 400000, ('Customer 4', 'Product 14', Timestamp('2025-05-25 00:00:00')): 1400000, ('Customer 4', 'Product 24', Timestamp('2025-05-25 00:00:00')): 600000, ('Customer 5', 'Product 7', Timestamp('2025-05-25 00:00:00')): 800000, ('Customer 6', 'Product 8', Timestamp('2025-05-25 

In [451]:
holding_cost = 0.001
M = 20 # penalizacao por falta a demanda
# truck_capacity = 250000
estoque_inicial_p = {i:0 for i in P}
estoque_inicial_w = {i:0 for i in W}

#Modelo

In [452]:
from pyomo.environ import *
model = ConcreteModel()

#inicializa os conjuntos
model.P = Set(initialize=P) # plantas
model.L = Set(initialize=Lines) # Linhas
model.W = Set(initialize=W) # armazens
model.C = Set(initialize=C) # clientes
model.I = Set(initialize=I) # produtos
model.T = Set(initialize=T) # periodos
model.E = Set(initialize=tamanhos) # tamanhos
model.Origins = model.W | model.P
model.Destinations = model.W | model.P | model.C

#inicializa os parâmetros
model.G_w = Param(model.W, initialize=G_w) # capacidade maxima de estoque no armzem w
model.G_p = Param(model.P, initialize=G_p) # capacidade maxima de estoque na planta p
model.h = Param(initialize=holding_cost) # custo de estoque por unidade por periodo
model.M = Param(initialize=M) # penalizador de demanda
# model.Q = Param(initialize=truck_capacity) # capacidade maxima por caminhao
model.k = Param(model.E,model.L,model.T, initialize=taxas, default= 0) # capacidade produtiva de cada linha
model.t_pp = Param(model.P, model.P, initialize=t_pp, default=1e20) # custos  de frete
model.t_pc = Param(model.P, model.C, initialize=t_pc, default=1e20) # custos de frete
model.t_wc = Param(model.W, model.C, initialize=t_wc, default=1e20) # custos de frete
model.t_pw = Param(model.P, model.W, initialize=t_pw, default=1e20) #custos de frete

# model.t_wp = Param(model.W, model.P, initialize=t_wp, default=1e20) #custos de frete
model.D = Param(model.C, model.I,model.T, initialize=D, default=0) # demandas
model.estoque_inicial_p = Param(model.P, initialize=estoque_inicial_p)
model.estoque_inicial_w = Param(model.W, initialize=estoque_inicial_w)
model.lead_time = Param(model.Origins, model.Destinations, initialize=ld, default = 20)

#incializar as variaveis

model.x = Var(model.I , model.L , model.T, domain=NonNegativeReals)
model.x_p = Var(model.I , model.P , model.T , domain=NonNegativeReals)
model.S_p = Var(model.I, model.P, model.T, domain=NonNegativeReals)
model.S_w = Var(model.I, model.W, model.T, domain=NonNegativeReals)
model.f_pc = Var(model.I, model.P, model.C, model.T, domain=NonNegativeReals)
model.f_wc = Var(model.I, model.W, model.C, model.T, domain=NonNegativeReals)
model.f_pp = Var(model.I, model.P, model.P, model.T ,domain=NonNegativeReals)
model.f_pw = Var(model.I, model.P, model.W, model.T  ,domain=NonNegativeReals)
model.f_wp = Var(model.I, model.W, model.P, model.T  ,domain=NonNegativeReals)
model.U = Var(model.C, model.I, model.T, domain=NonNegativeReals)

# Variaveis de numero de caminhoes enviados
model.y_pc = Var(model.P, model.C, model.T, domain=NonNegativeIntegers)
model.y_wc = Var(model.W, model.C, model.T, domain=NonNegativeIntegers)
model.y_pp = Var(model.P, model.P, model.T, domain=NonNegativeIntegers)
model.y_pw = Var(model.P, model.W, model.T, domain=NonNegativeIntegers)
model.y_wp = Var(model.W, model.P, model.T, domain=NonNegativeIntegers)

In [453]:
# definir funcao objetivo

def total_cost(m):
      # custo de frete Planta-Cliente
      cost_f_pc = sum(m.t_pc[p,c]* m.y_pc[p,c,t]
                  for p in m.P for c in m.C for t in m.T)
      # custo de frete Armazem-Cliente
      cost_f_wc = sum(m.t_wc[w, c] * m.y_wc[w, c, t]
                  for w in m.W for c in m.C for t in m.T)

      # custo de frete Planta-Planta
      cost_f_pp = sum(m.t_pp[p1, p2] * m.y_pp[p1, p2, t]
                  for p1 in m.P for p2 in m.P if p1 != p2 for t in m.T) # Adicionado if p1 != p2 para evitar frete de planta para ela mesma

      # custo de frete Planta-Armazem e Armazem-Planta
      cost_f_pw_wp = sum(m.t_pw[p, w] * (m.y_pw[p, w, t] + m.y_wp[w,p,t])
                  for p in m.P for w in m.W for t in m.T)

      # custo de estoque no Armazem
      cost_h_w = sum(m.h * m.S_w[i,w,t]
                  for i in m.I for w in m.W for t in m.T)

      # custo de penalidade por demanda nao atendida
      cost_unmet_demand = m.M * sum(m.U[c,i,t]
                                    for c in m.C for i in m.I for t in m.T)

      return cost_f_pc + cost_f_wc + cost_f_pp + cost_f_pw_wp + cost_h_w + cost_unmet_demand


model.obj = Objective(rule=total_cost, sense=minimize)


In [454]:
# definir as restrições

# 1) restricao de producao
model.Producao = ConstraintList()
for t in model.T:
  for l in model.L:
      for e in model.E:
        model.Producao.add(expr = sum(model.x[i,l,t] for i in model.I if size_of_products.get(i) == e) <= model.k[e,l,t])

# Restrição para definir x_p como a soma da produção por planta
model.Definicao_x_p = ConstraintList()
for i in model.I:
  for p in model.P:
    for t in model.T:
      model.Definicao_x_p.add(expr = model.x_p[i, p, t] == sum(model.x[i, l, t] for l in model.L if p_l[l] == p))



# 2) restricao de balanco de estoque na planta
# Para o primeiro período (t=1), consideramos o estoque inicial como 0
model.Balanco_estoque_planta = ConstraintList()
for p in model.P:
  for i in model.I:
    # Assuming T starts from 1
    first_period = min(model.T)
    model.Balanco_estoque_planta.add(
        expr = model.estoque_inicial_p[p] + model.x_p[i, p, first_period] + sum(model.f_pp[i, q, p, T[period_index[first_period]- model.lead_time[q,p]]] for q in model.P if q != p and period_index[first_period] - model.lead_time[q,p]>=0)
        + sum(model.f_wp[i, w, p, T[period_index[first_period] - model.lead_time[w,p]]] for w in model.W if period_index[first_period] - model.lead_time[w,p]>=0)==
               model.S_p[i, p, first_period] + sum(model.f_pw[i, p, w, first_period] for w in model.W) + sum(model.f_pc[i, p, c, first_period] for c in model.C) + sum(model.f_pp[i, p, q, first_period] for q in model.P if q != p)
    )
# Para os períodos subsequentes (t > 1)
    for t in model.T:
      if t > min(model.T):
        previous_period = T[period_index[t] - 1]
        model.Balanco_estoque_planta.add(
            expr = model.S_p[i, p, previous_period] + model.x_p[i, p, t] + sum(model.f_pp[i, q, p, T[period_index[t]- model.lead_time[q,p]]] for q in model.P if q != p and period_index[t] - model.lead_time[q,p] >= 0)
            + sum(model.f_wp[i,w,p,T[period_index[t] - model.lead_time[w,p]]] for w in model.W if period_index[t] - model.lead_time[w,p] >=0) ==
                   model.S_p[i, p, t] + sum(model.f_pw[i, p, w, t] for w in model.W) + sum(model.f_pc[i, p, c, t] for c in model.C) + sum(model.f_pp[i, p, q, t] for q in model.P if q != p)
        )


# 3)restricao de balanco de estoque no armazem
model.Balanco_estoque_armazem = ConstraintList()
first_period = min(model.T)
for w in model.W:
  for i in model.I:
    model.Balanco_estoque_armazem.add(
        expr =  model.estoque_inicial_w[w]
        + sum(model.f_pw[i, p, w, T[period_index[first_period] - model.lead_time[p,w]]] for p in model.P if period_index[first_period] - model.lead_time[p,w] >= 0)
        == model.S_w[i, w, first_period] + sum(model.f_wp[i, w, p, first_period] for p in model.P) +sum(model.f_wc[i, w, c, first_period] for c in model.C)
    )
# Para os períodos subsequentes (t > 1)
    for t in model.T:
      if t > min(model.T):
        previous_period = T[period_index[t] - 1]
        model.Balanco_estoque_armazem.add(
            expr = model.S_w[i, w, previous_period] + sum(model.f_pw[i, p, w, T[period_index[t] - model.lead_time[p,w]]] for p in model.P if period_index[t]-model.lead_time[p,w] >= 0) ==
                   model.S_w[i, w, t] + sum(model.f_wc[i, w, c, t] for c in model.C) + sum(model.f_wp[i, w, p, t] for p in model.P)
        )

# 4) restricao de estoque maximo na planta
model.Estoque_maximo_p = ConstraintList()
for p in model.P:
  for t in model.T:
    model.Estoque_maximo_p.add(
        expr = sum(model.S_p[i, p, t] for i in model.I) <= model.G_p[p]
    )


# 5) restricao de estoque maximo no armazem
model.Estoque_maximo_w = ConstraintList()
for w in model.W:
  for t in model.T:
    model.Estoque_maximo_w.add(
        expr = sum(model.S_w[i, w, t] for i in model.I) <= model.G_w[w]
    )


# 6) restricao de demanda
model.Demanda = ConstraintList()
for c in model.C:
  for i in model.I:
    for t in model.T:
      model.Demanda.add(
          expr = sum(model.f_pc[i, p, c, T[period_index[t] - model.lead_time[p,c]]] for p in model.P if period_index[t] - model.lead_time[p,c] >= 0) + sum(model.f_wc[i, w, c, T[period_index[t] - model.lead_time[w,c]]] for w in model.W if period_index[t]- model.lead_time[w,c] >= 0) + model.U[c,i,t] >= model.D[c, i, t])


# restricao para determinar numero de caminhoes

model.Caminhoes = ConstraintList()
for t in model.T:
  for p in model.P:
    for w in model.W:
      model.Caminhoes.add(expr=sum(model.f_pw[i,p,w,t] for i in model.I) <= model.y_pw[p,w,t] * 250000) 
      
    for c in model.C:
      model.Caminhoes.add(expr=sum(model.f_pc[i,p,c,t] for i in model.I) <= model.y_pc[p,c,t] * 250000)

  for w in model.W:
    for p in model.P:
      model.Caminhoes.add(expr=sum(model.f_wp[i,w,p,t] for i in model.I) <= model.y_wp[w,p,t] * 250000) 
    
    for c in model.C:
      model.Caminhoes.add(expr=sum(model.f_wc[i,w,c,t] for i in model.I) <= model.y_wc[w,c,t] * 250000)
      



In [455]:
import gurobipy

In [456]:
solver = SolverFactory('gurobi')

solver.solve(model)
# Obtém o valor do objetivo
objective_value = model.obj()
print(f"Valor do objetivo : {objective_value:.2f} ")

Valor do objetivo : 5197839.83 


In [457]:
# Imprimir o valor das variáveis U[c,i,t]
unmet_demand_total = 0
unmet_demands = {}
u_df = {"Customer":[], "Product":[],"Period":[],"Unmet Demand":[]}
for c in model.C:
  for i in model.I:
    for t in model.T:
      U = model.U[c,i,t].value
      unmet_demand_total += U
      unmet_demands[(c,i,t)] = U
      # print(f"U[{c},{i},{t}] = {U}")
      if U >0:
        u_df["Customer"].append(c)
        u_df["Product"].append(i)
        u_df["Period"].append(t)
        u_df["Unmet Demand"].append(U)

print(f"Total unmet demand: {unmet_demand_total}")
pct_met_demand = ((total_demand - unmet_demand_total)/total_demand) * 100
print(f"Total cost of operation: {objective_value - unmet_demand_total*M}")
print(f"Percentage of demand met: {pct_met_demand:.2f}%")

Total unmet demand: 0.0
Total cost of operation: 5197839.826999995
Percentage of demand met: 100.00%


In [458]:
print("Unmet Demands: ")
unmet_demands
u_df = pd.DataFrame(u_df)
u_df

Unmet Demands: 


,Customer,Product,Period,Unmet Demand


In [459]:
# Create dictionaries to store variable values
x_data = {"Line": [], "Period": [],"Product": [], "Value": []}
x_p_data = { "Plant": [], "Period": [],"Product": [], "Value": []}
S_p_data = { "Plant": [], "Period": [],"Product": [], "Value": []}
S_w_data = { "Warehouse": [], "Period": [],"Product": [], "Value": []}
f_pc_data = {"Plant": [], "Period": [], "Customer": [], "Product": [], "Value": []}
f_wc_data = {"Warehouse": [], "Period": [], "Customer": [], "Product": [], "Value": []}
f_wp_data = {"Warehouse": [], "Period": [], "Plant": [], "Product": [], "Value": []}
f_pp_data = {"Plant_Origin": [], "Period": [], "Plant_Destination": [], "Product": [], "Value": []}
f_pw_data = {"Plant": [], "Period": [], "Warehouse": [], "Product": [], "Value": []}
u_df = {"Period": [], "Customer" : [],"Product" : [], "Unmet Demand" : []}
y_pc_data = {"Plant": [], "Period": [], "Customer": [], "Trucks": []}
y_wc_data = {"Warehouse": [], "Period": [], "Customer": [], "Trucks": []}
y_wp_data = {"Warehouse": [], "Period": [], "Plant": [], "Trucks": []}
y_pp_data = {"Plant_Origin": [], "Period": [], "Plant_Destination": [], "Trucks": []}
y_pw_data = {"Plant": [], "Period": [], "Warehouse": [], "Trucks": []}


freight_cost_total = 0

stock_cost_total = 0

# Extract values from each variable
for l in model.L:
  for t in model.T:
    for i in model.I:
      prod = model.x[i, l, t].value
      if prod > 0:
        x_data["Line"].append(l)
        x_data["Period"].append(t)
        x_data["Product"].append(i)
        x_data["Value"].append(prod)

for p in model.P:
  for t in model.T:
    for i in model.I:

      prod = model.x_p[i, p, t].value
      if prod > 0:
        x_p_data["Plant"].append(p)
        x_p_data["Period"].append(t)
        x_p_data["Product"].append(i)
        x_p_data["Value"].append(prod)

      stock = model.S_p[i, p, t].value
      if stock > 0:
        S_p_data["Plant"].append(p)
        S_p_data["Period"].append(t)
        S_p_data["Product"].append(i)
        S_p_data["Value"].append(stock)



for w in model.W:
  for t in model.T:
    
    for i in model.I:
      stock = model.S_w[i, w, t].value
      if stock > 0:
        S_w_data["Warehouse"].append(w)
        S_w_data["Period"].append(t)
        S_w_data["Product"].append(i)
        S_w_data["Value"].append(stock)
        stock_cost_total+= stock*model.h

    for c in model.C:

      yflow = model.y_wc[w, c, t].value
      if yflow > 0:
        y_wc_data["Warehouse"].append(w)
        y_wc_data["Period"].append(t)
        y_wc_data["Customer"].append(c)
        y_wc_data["Trucks"].append(yflow)
        freight_cost_total+= yflow*model.t_wc[w,c]

      for i in model.I:

        flow = model.f_wc[i, w, c, t].value
        if flow > 0:
          f_wc_data["Warehouse"].append(w)
          f_wc_data["Period"].append(t)
          f_wc_data["Customer"].append(c)
          f_wc_data["Product"].append(i)
          f_wc_data["Value"].append(flow)




    for p in model.P:

      yflow = model.y_wp[w, p, t].value
      if yflow > 0:
        y_wp_data["Warehouse"].append(w)
        y_wp_data["Period"].append(t)
        y_wp_data["Plant"].append(p)
        y_wp_data["Trucks"].append(yflow)
        freight_cost_total+= yflow*model.t_pw[p,w]        

      for i in model.I:

        flow = model.f_wp[i, w, p, t].value
        if flow > 0:
          f_wp_data["Warehouse"].append(w)
          f_wp_data["Period"].append(t)
          f_wp_data["Plant"].append(p)
          f_wp_data["Product"].append(i)
          f_wp_data["Value"].append(flow)


for p in model.P:
  for t in model.T:
    for c in model.C:


      yflow = model.y_pc[p, c, t].value
      if yflow > 0:
        y_pc_data["Plant"].append(p)
        y_pc_data["Period"].append(t)
        y_pc_data["Customer"].append(c)
        y_pc_data["Trucks"].append(yflow)
        freight_cost_total+= yflow*model.t_pc[p,c]

      for i in model.I:

        flow = model.f_pc[i, p, c, t].value
        if flow > 0:
          f_pc_data["Plant"].append(p)
          f_pc_data["Period"].append(t)
          f_pc_data["Customer"].append(c)
          f_pc_data["Product"].append(i)
          f_pc_data["Value"].append(flow)

    for p2 in model.P:
      if p != p2: # Assuming f_pp is only for flow between different plants

        yflow = model.y_pp[p, p2, t].value
        if yflow > 0:
          y_pp_data["Plant_Origin"].append(p)
          y_pp_data["Period"].append(t)
          y_pp_data["Plant_Destination"].append(p2)
          y_pp_data["Trucks"].append(yflow)
          freight_cost_total+= yflow*model.t_pp[p,p2]

        for i in model.I:

          flow = model.f_pp[i, p, p2, t].value
          if flow > 0:
            f_pp_data["Plant_Origin"].append(p)
            f_pp_data["Period"].append(t)
            f_pp_data["Plant_Destination"].append(p2)
            f_pp_data["Product"].append(i)
            f_pp_data["Value"].append(flow)


    for w in model.W:

      yflow = model.y_pw[p, w, t].value
      if yflow > 0:
        y_pw_data["Plant"].append(p)
        y_pw_data["Period"].append(t)
        y_pw_data["Warehouse"].append(w)
        y_pw_data["Trucks"].append(yflow)
        freight_cost_total+= yflow*model.t_pw[p,w]

      for i in model.I:

        flow = model.f_pw[i, p, w, t].value
        if flow > 0:
          f_pw_data["Plant"].append(p)
          f_pw_data["Period"].append(t)
          f_pw_data["Warehouse"].append(w)
          f_pw_data["Product"].append(i)
          f_pw_data["Value"].append(flow)


# print("Valores das variáveisc U[c,i,t]:")
for c in model.C:
  for t in model.T:
    for i in model.I:

      unmet = model.U[c,i,t].value
      if unmet > 0:
        u_df["Customer"].append(c)
        u_df["Period"].append(t)
        u_df["Product"].append(i)
        u_df["Unmet Demand"].append(unmet)


# Create DataFrames
x_df = pd.DataFrame(x_data)
x_p_df = pd.DataFrame(x_p_data)
S_p_df = pd.DataFrame(S_p_data)
S_w_df = pd.DataFrame(S_w_data)
f_pc_df = pd.DataFrame(f_pc_data)
f_wc_df = pd.DataFrame(f_wc_data)
f_wp_df = pd.DataFrame(f_wp_data)
f_pp_df = pd.DataFrame(f_pp_data)
f_pw_df = pd.DataFrame(f_pw_data)
u_df = pd.DataFrame(u_df)
y_pc_df = pd.DataFrame(y_pc_data)
y_wc_df = pd.DataFrame(y_wc_data)
y_wp_df = pd.DataFrame(y_wp_data)
y_pp_df = pd.DataFrame(y_pp_data)
y_pw_df = pd.DataFrame(y_pw_data)

# Write to Excel
with pd.ExcelWriter('./results/decision_variables2.xlsx') as writer:
    x_df.to_excel(writer, sheet_name='x', index=False)
    x_p_df.to_excel(writer, sheet_name='x_p', index=False)
    S_p_df.to_excel(writer, sheet_name='S_p', index=False)
    S_w_df.to_excel(writer, sheet_name='S_w', index=False)
    f_pc_df.to_excel(writer, sheet_name='f_pc', index=False)
    f_wc_df.to_excel(writer, sheet_name='f_wc', index=False)
    f_wp_df.to_excel(writer, sheet_name='f_wp', index=False)
    f_pp_df.to_excel(writer, sheet_name='f_pp', index=False)
    f_pw_df.to_excel(writer, sheet_name='f_pw', index=False)
    y_pc_df.to_excel(writer, sheet_name='y_pc', index=False)
    y_wc_df.to_excel(writer, sheet_name='y_wc', index=False)
    y_wp_df.to_excel(writer, sheet_name='y_wp', index=False)
    y_pp_df.to_excel(writer, sheet_name='y_pp', index=False)
    y_pw_df.to_excel(writer, sheet_name='y_pw', index=False)
    u_df.to_excel(writer,sheet_name='U', index=False)

In [460]:
print(stock_cost_total)
print(freight_cost_total)

1400.0
5196439.826999998
